In [7]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlalchemy
import pymysql.cursors
import os
import yaml


## 1. Data Cleaning

### Data Cleaning of the main dataframe (Sorare Cards Transactions)

#### a.  Sorare Transactions Loading & bird's view


In [8]:
all_transactions_df = pd.read_csv('all_players_cards_transactions_la_liga.csv', dtype={20: str})

In [9]:
all_transactions_df.shape

(1741383, 21)

In [10]:
all_transactions_df.dtypes

team_name              object
card_id                object
asset_id               object
card_name              object
season_year             int64
card_slug              object
rarity                 object
card_creation_date     object
grade                   int64
power                 float64
xp                      int64
serial_number           int64
shirt_number            int64
special_edition        object
transaction_date       object
transaction_type       object
price_wei               int64
price_eur             float64
user_id                object
user_slug              object
player_slug            object
dtype: object

In [11]:
all_transactions_df.head()

,team_name,card_id,asset_id,card_name,season_year,card_slug,rarity,card_creation_date,grade,power,...,serial_number,shirt_number,special_edition,transaction_date,transaction_type,price_wei,price_eur,user_id,user_slug,player_slug
0,Real Madrid CF,Card:00035848-b60d-4999-832c-e4dd8e939007,0x040004752c83837b25107500c71efa485aafb6cf1b82...,Nacho 2021-22 • Limited 218/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-218,limited,2021-11-25T03:40:39Z,7,1.045,...,218,6,NaN,2022-01-18T10:23:04Z,REFERRAL,0,0.000000,User:dff07274-2664-4ed9-9308-4de48a2b8bec,serveg,jose-ignacio-fernandez-iglesias
1,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31T08:58:22Z,13,1.075,...,488,6,NaN,2022-07-19T09:23:20Z,ENGLISH_AUCTION,6500000000000000,9.747140,User:ee426172-1955-4317-98d7-480476d0bb28,anonymous-8f8df118-a1d8-493d-9a5e-98997ed6450b,jose-ignacio-fernandez-iglesias
2,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31T08:58:22Z,13,1.075,...,488,6,NaN,2022-07-19T09:36:47Z,SINGLE_SALE_OFFER,5100000000000000,7.647756,User:f831ecc8-990a-42e7-94b5-6656536685f5,bugggyman-ua,jose-ignacio-fernandez-iglesias
3,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31T08:58:22Z,13,1.075,...,488,6,NaN,2022-07-19T11:55:20Z,SINGLE_SALE_OFFER,5400000000000000,8.116470,User:9d7ae8cb-c48e-4e5b-8715-eae0826c06c1,junmisugi84-e6f8dfdd-08b3-463f-a2b5-d5870eb74deb,jose-ignacio-fernandez-iglesias
4,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31T08:58:22Z,13,1.075,...,488,6,NaN,2023-03-14T10:28:55Z,SINGLE_BUY_OFFER,2800000000000000,4.356016,User:2d9f44fb-d504-4159-96bd-89deaa61e585,alvaropm019,jose-ignacio-fernandez-iglesias


#### b. removing null values

In [12]:
all_transactions_df.isna().sum()

team_name                   0
card_id                     0
asset_id                    0
card_name                   0
season_year                 0
card_slug                   0
rarity                      0
card_creation_date          0
grade                       0
power                       0
xp                          0
serial_number               0
shirt_number                0
special_edition       1503306
transaction_date            0
transaction_type            0
price_wei                   0
price_eur                   0
user_id                     0
user_slug                   0
player_slug             61549
dtype: int64

In [13]:
#1 for "special edition" replacing values: 
## - all_transactions_df['special_edition'].value_counts() shows that "classic" edition appears recently and is not a "special edition" attribute. 
## - handled by putting "classic" and none values to "False" and others to "True" and convert to boolean. 


all_transactions_df['special_edition'] = all_transactions_df['special_edition'].fillna('unknown')

# Replace 'classic' with 'False' and all other values with 'True'
all_transactions_df['special_edition'] = all_transactions_df['special_edition'].replace({'classic': False})
all_transactions_df['special_edition'] = all_transactions_df['special_edition'].replace({'unknown': False}, regex=True)
all_transactions_df['special_edition'] = all_transactions_df['special_edition'].replace({'.*': True}, regex=True)

# Convert the column to boolean type
all_transactions_df['special_edition'] = all_transactions_df['special_edition'].astype(bool)

In [14]:
all_transactions_df['special_edition'].value_counts()

special_edition
False    1693790
True       47593
Name: count, dtype: int64

In [16]:
# removing the null values in "player_slug"
all_transactions_df.dropna(subset=['player_slug'], inplace=True)

#### c. Parsing and computing the dates: 

In [17]:
def time_computation(df, column):
    df[column] = pd.to_datetime(df[column])
    df[f'{column}_year'] = df[column].dt.year
    df[f'{column}_month'] = df[column].dt.month
    df[f'{column}_week'] = df[column].dt.isocalendar().week
    df[f'{column}_dayofyear'] = df[column].dt.dayofyear
    df[f'{column}_season'] = np.where(df[column].dt.month >= 8, df[f'{column}_year'], df[f'{column}_year'] - 1)
    df[f'{column}_season'] = 'season_' + df[f'{column}_season'].astype(str)
    return df

all_transactions_df = time_computation(all_transactions_df, 'card_creation_date')
all_transactions_df = time_computation(all_transactions_df, 'transaction_date')

In [18]:
all_transactions_df.head()

,team_name,card_id,asset_id,card_name,season_year,card_slug,rarity,card_creation_date,grade,power,...,card_creation_date_year,card_creation_date_month,card_creation_date_week,card_creation_date_dayofyear,card_creation_date_season,transaction_date_year,transaction_date_month,transaction_date_week,transaction_date_dayofyear,transaction_date_season
0,Real Madrid CF,Card:00035848-b60d-4999-832c-e4dd8e939007,0x040004752c83837b25107500c71efa485aafb6cf1b82...,Nacho 2021-22 • Limited 218/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-218,limited,2021-11-25 03:40:39+00:00,7,1.045,...,2021,11,47,329,season_2021,2022,1,3,18,season_2021
1,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31 08:58:22+00:00,13,1.075,...,2022,5,22,151,season_2021,2022,7,29,200,season_2021
2,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31 08:58:22+00:00,13,1.075,...,2022,5,22,151,season_2021,2022,7,29,200,season_2021
3,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31 08:58:22+00:00,13,1.075,...,2022,5,22,151,season_2021,2022,7,29,200,season_2021
4,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31 08:58:22+00:00,13,1.075,...,2022,5,22,151,season_2021,2023,3,11,73,season_2022


#### d. Removing transactions when players were not in a la liga clubs

In [ ]:
# Removing special cards that don't are interesting in the 

In [19]:
# list of slugs and names of la liga clubs since 2021: 

list_of_team_slugs = ["real-madrid-madrid",
                 "girona-girona",
                 "barcelona-barcelona", 
                 "atletico-madrid-madrid",
                 "athletic-club-bilbao",
                 "real-sociedad-donostia-san-sebastian", 
                 "valencia-valencia", 
                 "real-betis-sevilla",
                 "las-palmas-las-palmas-de-gran-canaria",
                "getafe-getafe-madrid",
                 "deportivo-alaves-vitoria-gasteiz",
                 "osasuna-pamplona-irunea",
                 "rayo-vallecano-madrid",
                 "villarreal-villarreal",
                 "sevilla-sevilla-1890",
                 "celta-de-vigo-vigo",
                 "mallorca-palma-de-mallorca",
                 "cadiz-cadiz",
                 "granada-granada",
                 "almeria-almeria",
                 "real-valladolid-valladolid",
                 "espanyol-barcelona",
                 "elche-elche",
                 "levante-valencia"
                ]

list_of_team_names = ['Real Madrid CF',
                 'Girona FC',
                 'FC Barcelona',
                 'Club Atlético de Madrid',
                 'Athletic Club Bilbao',
                 'Real Sociedad de Fútbol',
                 'Valencia CF',
                 'Real Betis Balompié',
                 'UD Las Palmas',
                 'Getafe Club de Fútbol',
                 'Deportivo Alavés',
                 'CA Osasuna',
                 'Rayo Vallecano',
                 'Villarreal CF',
                 'Sevilla FC',
                 'Real Club Celta de Vigo',
                 'Real Club Deportivo Mallorca',
                 'Cádiz CF',
                 'Granada CF',
                 'UD Almería',
                 'Real Valladolid Club de Fútbol',
                 'Reial Club Deportiu Espanyol',
                 'Elche CF',
                 'Levante UD']

In [20]:
#loading the clubs history of players

players_club_hist_df = pd.read_csv('clubs_hist.csv')

In [21]:
players_club_hist_df.dtypes

player_slug            object
position               object
age                     int64
u23_eligible             bool
u23_until              object
club_slug              object
contract_start_date    object
contract_end_date      object
dtype: object

In [22]:
#parsing the date: 
players_club_hist_df['contract_start_date'] = pd.to_datetime(players_club_hist_df['contract_start_date'])
players_club_hist_df['contract_end_date'] = pd.to_datetime(players_club_hist_df['contract_end_date'])

In [23]:


# Step 1: Filter players_club_hist_df for La Liga club history
la_liga_clubs_df = players_club_hist_df[players_club_hist_df['club_slug'].isin(list_of_team_slugs)]

# Step 2: Merge all_transactions_df with la_liga_clubs_df based on player_slug and contract dates
merged_df = pd.merge(all_transactions_df, la_liga_clubs_df, on='player_slug', how='inner')
filtered_transactions_df = merged_df[
    (merged_df['transaction_date'] >= merged_df['contract_start_date']) &
    (merged_df['transaction_date'] <= merged_df['contract_end_date'])
].copy() # Create a copy of the DataFrame

# Step 3: Rename columns to avoid confusion
filtered_transactions_df.rename(columns={'club_slug': 'club_slug_upon_transaction', 'team_name': 'present_team_name'}, inplace=True)

# Step 4: Drop unnecessary columns
filtered_transactions_df = filtered_transactions_df.drop(columns=['contract_start_date', 'contract_end_date'])



In [24]:
filtered_transactions_df.shape

(1349824, 36)

In [25]:
filtered_transactions_df['club_slug_upon_transaction'].value_counts()

club_slug_upon_transaction
real-madrid-madrid                       89095
sevilla-sevilla-1890                     80973
real-betis-sevilla                       79597
atletico-madrid-madrid                   74357
villarreal-villarreal                    74219
barcelona-barcelona                      67493
athletic-club-bilbao                     66256
osasuna-pamplona-irunea                  63721
mallorca-palma-de-mallorca               60937
real-sociedad-donostia-san-sebastian     59829
celta-de-vigo-vigo                       59657
rayo-vallecano-madrid                    59327
valencia-valencia                        57020
getafe-getafe-madrid                     55180
espanyol-barcelona                       54783
deportivo-alaves-vitoria-gasteiz         53073
granada-granada                          49234
cadiz-cadiz                              47982
levante-valencia                         46221
elche-elche                              44190
girona-girona                    

In [26]:
""" tests to check if it works well. VALIDATED all_rulli_transactions = all_transactions_df[all_transactions_df["player_slug"] == "geronimo-rulli"]
all_rulli_transactions.shape

filt_rulli_transactions = filtered_transactions_df[filtered_transactions_df["player_slug"] == "geronimo-rulli"]
filt_rulli_transactions.shape

all_jude_transactions = all_transactions_df[all_transactions_df["player_slug"] == "jude-bellingham"]
all_jude_transactions.shape

filt_jude_transactions = filtered_transactions_df[filtered_transactions_df["player_slug"] == "jude-bellingham"]
filt_jude_transactions.shape
"""

' tests to check if it works well. VALIDATED all_rulli_transactions = all_transactions_df[all_transactions_df["player_slug"] == "geronimo-rulli"]\nall_rulli_transactions.shape\n\nfilt_rulli_transactions = filtered_transactions_df[filtered_transactions_df["player_slug"] == "geronimo-rulli"]\nfilt_rulli_transactions.shape\n\nall_jude_transactions = all_transactions_df[all_transactions_df["player_slug"] == "jude-bellingham"]\nall_jude_transactions.shape\n\nfilt_jude_transactions = filtered_transactions_df[filtered_transactions_df["player_slug"] == "jude-bellingham"]\nfilt_jude_transactions.shape\n'

#### c. Deal with club_names different input during fetching (replaces slugs by names)

In [27]:
replacement_dict = dict(zip(list_of_team_slugs, list_of_team_names))

filtered_transactions_df['present_team_name'] = filtered_transactions_df['present_team_name'].replace(replacement_dict)

#### d. Filter only on transactions where promoted or relegated were in La Liga

In [28]:
clubs_df = pd.DataFrame({'team_name': list_of_team_names, 'team_slug': list_of_team_slugs})
clubs_df

,team_name,team_slug
0,Real Madrid CF,real-madrid-madrid
1,Girona FC,girona-girona
2,FC Barcelona,barcelona-barcelona
3,Club Atlético de Madrid,atletico-madrid-madrid
4,Athletic Club Bilbao,athletic-club-bilbao
5,Real Sociedad de Fútbol,real-sociedad-donostia-san-sebastian
6,Valencia CF,valencia-valencia
7,Real Betis Balompié,real-betis-sevilla
8,UD Las Palmas,las-palmas-las-palmas-de-gran-canaria
9,Getafe Club de Fútbol,getafe-getafe-madrid


In [29]:
##clubs_df.to_csv("clubs_by_season.csv", index=False)
clubs_by_season_df = pd.read_csv('clubs_by_season.csv', sep=';')
clubs_by_season_df 

,team_name,team_slug,2021_22_la_liga,2022_23_la_liga,2023_24_la_liga,2021_22_uefa,2022_23_uefa,2023_24_uefa
0,Real Madrid CF,real-madrid-madrid,True,True,True,True,True,True
1,Girona FC,girona-girona,False,True,True,False,False,False
2,FC Barcelona,barcelona-barcelona,True,True,True,True,True,True
3,Club Atlético de Madrid,atletico-madrid-madrid,True,True,True,True,True,True
4,Athletic Club Bilbao,athletic-club-bilbao,True,True,True,False,False,False
5,Real Sociedad de Fútbol,real-sociedad-donostia-san-sebastian,True,True,True,True,True,True
6,Valencia CF,valencia-valencia,True,True,True,False,False,False
7,Real Betis Balompié,real-betis-sevilla,True,True,True,True,True,True
8,UD Las Palmas,las-palmas-las-palmas-de-gran-canaria,True,False,True,False,False,False
9,Getafe Club de Fútbol,getafe-getafe-madrid,True,True,True,False,False,False


In [30]:
clubs_by_season_df['2021_22_la_liga'] = clubs_by_season_df['2021_22_la_liga'].astype(bool)
clubs_by_season_df['2022_23_la_liga'] = clubs_by_season_df['2022_23_la_liga'].astype(bool)
clubs_by_season_df['2023_24_la_liga'] = clubs_by_season_df['2023_24_la_liga'].astype(bool)
clubs_by_season_df['2021_22_uefa'] = clubs_by_season_df['2021_22_uefa'].astype(bool)
clubs_by_season_df['2022_23_uefa'] = clubs_by_season_df['2022_23_uefa'].astype(bool)
clubs_by_season_df['2023_24_uefa'] = clubs_by_season_df['2023_24_uefa'].astype(bool)
clubs_by_season_df.dtypes

team_name          object
team_slug          object
2021_22_la_liga      bool
2022_23_la_liga      bool
2023_24_la_liga      bool
2021_22_uefa         bool
2022_23_uefa         bool
2023_24_uefa         bool
dtype: object

In [31]:
# Droping the transactions of clubs not in la liga for a specific season (not yet promoted or relegated)


#Create a copy of the filtered_transactions_df to avoid overwriting it
la_liga_transactions_df = filtered_transactions_df.copy()

# Define the list of seasons

seasons = [
    {'name': '2021_22_la_liga', 'start_date': '2021-09-01', 'end_date': '2022-06-30'},
    {'name': '2022_23_la_liga', 'start_date': '2022-09-01', 'end_date': '2023-06-30'},
    {'name': '2023_24_la_liga', 'start_date': '2023-09-01', 'end_date': '2024-06-30'}
]

# Merge la_liga_transactions_df with clubs_by_season based on both club_slug_upon_transaction and team_slug
la_liga_transactions_df = pd.merge(
        la_liga_transactions_df,
        clubs_by_season_df,
        left_on=['club_slug_upon_transaction'],
        right_on=['team_slug'],
        how='inner'
    )

# Loop over each season
for season in seasons:
    # Filter clubs_out_season based on the current season and team slug
    season_column = season['name']
    clubs_out_season = clubs_by_season_df[clubs_by_season_df[season_column] == False]['team_slug']

    # Define the date range for the current season
    start_date = season['start_date']
    end_date = season['end_date']

    # Drop transactions within the date range and matching clubs_out_season
    la_liga_transactions_df.drop(
        la_liga_transactions_df[
            (la_liga_transactions_df['club_slug_upon_transaction'].isin(clubs_out_season)) &
            (la_liga_transactions_df['transaction_date'].between(start_date, end_date))
        ].index,
        inplace=True
    )

In [32]:
filtered_transactions_df.shape

(1349824, 36)

In [33]:
la_liga_transactions_df.shape

(1261014, 44)

In [34]:
la_liga_transactions_df.head()

,present_team_name,card_id,asset_id,card_name,season_year,card_slug,rarity,card_creation_date,grade,power,...,u23_until,club_slug_upon_transaction,team_name,team_slug,2021_22_la_liga,2022_23_la_liga,2023_24_la_liga,2021_22_uefa,2022_23_uefa,2023_24_uefa
0,Real Madrid CF,Card:00035848-b60d-4999-832c-e4dd8e939007,0x040004752c83837b25107500c71efa485aafb6cf1b82...,Nacho 2021-22 • Limited 218/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-218,limited,2021-11-25 03:40:39+00:00,7,1.045,...,NaN,real-madrid-madrid,Real Madrid CF,real-madrid-madrid,True,True,True,True,True,True
1,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31 08:58:22+00:00,13,1.075,...,NaN,real-madrid-madrid,Real Madrid CF,real-madrid-madrid,True,True,True,True,True,True
2,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31 08:58:22+00:00,13,1.075,...,NaN,real-madrid-madrid,Real Madrid CF,real-madrid-madrid,True,True,True,True,True,True
3,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31 08:58:22+00:00,13,1.075,...,NaN,real-madrid-madrid,Real Madrid CF,real-madrid-madrid,True,True,True,True,True,True
4,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31 08:58:22+00:00,13,1.075,...,NaN,real-madrid-madrid,Real Madrid CF,real-madrid-madrid,True,True,True,True,True,True


In [35]:
""" Used for checking drops magnitudes

# Filtered clubs
concerned_clubs = clubs_by_season_df[clubs_by_season_df['2023_24_la_liga'] == False]['team_slug']

# Filtered transactions per quarter
filtered_transactions_per_quarter = filtered_transactions_df[
    filtered_transactions_df['club_slug_upon_transaction'].isin(concerned_clubs)
].groupby(['transaction_date_year','transaction_date_quarter']).count()

# Display the transactions per quarter
filtered_transactions_per_quarter

"""

" Used for checking drops magnitudes\n\n# Filtered clubs\nconcerned_clubs = clubs_by_season_df[clubs_by_season_df['2023_24_la_liga'] == False]['team_slug']\n\n# Filtered transactions per quarter\nfiltered_transactions_per_quarter = filtered_transactions_df[\n    filtered_transactions_df['club_slug_upon_transaction'].isin(concerned_clubs)\n].groupby(['transaction_date_year','transaction_date_quarter']).count()\n\n# Display the transactions per quarter\nfiltered_transactions_per_quarter\n\n"

In [36]:
la_liga_transactions_df['u23_until'] = la_liga_transactions_df['u23_until'].fillna('NA')

In [37]:
#la_liga_transactions_df.to_csv('la_liga_filtered_cards_transactions.csv', index=False)

#### f. Handling potential duplicates: 

In [95]:
la_liga_transactions_df.isna().sum()

present_team_name               0
card_id                         0
asset_id                        0
card_name                       0
season_year                     0
card_slug                       0
rarity                          0
card_creation_date              0
grade                           0
power                           0
xp                              0
serial_number                   0
shirt_number                    0
special_edition                 0
transaction_date                0
transaction_type                0
price_wei                       0
price_eur                       0
user_id                         0
user_slug                       0
player_slug                     0
card_creation_date_year         0
card_creation_date_month        0
card_creation_date_week         0
card_creation_date_dayofyear    0
card_creation_date_season       0
transaction_date_year           0
transaction_date_month          0
transaction_date_week           0
transaction_da

In [39]:
la_liga_transactions_df.duplicated().sum()

0

In [40]:
# having not fetch transaction ID cannot assess if some are duplicated. Apparently not. 
la_liga_transactions_df.duplicated(subset=['card_id']).sum()

786659

#### Cleaning/preparing the Prices 

In [41]:
# Converting WEI in ETH (ETH prices are in WEI, which 1e-18 ETH)
la_liga_transactions_df['price_eth'] = la_liga_transactions_df['price_wei'] * 1e-18

##### categorisation of transactions
There are both paying and free transactions (REWARD,REFERRAL) as well as non usable paying transaction (cards bought in bundles or packs).
Also, in the analysis, it will be insightful to distinguish primary marked (english_auction and more recently intant_buy) and secondary_market (_OFFER). 

In [42]:
la_liga_transactions_df['transaction_type'].value_counts()

transaction_type
SINGLE_SALE_OFFER          456314
ENGLISH_AUCTION            320100
DIRECT_OFFER               214920
SINGLE_BUY_OFFER           134390
REWARD                     111796
INSTANT_BUY                 10775
REFERRAL                     9625
BUNDLED_ENGLISH_AUCTION      1693
PACK                         1301
TRANSFER                       68
MINT                           26
DEPOSIT                         4
WITHDRAWAL                      2
Name: count, dtype: int64

In [43]:
# Create the transaction_category column
conditions = [
    la_liga_transactions_df['transaction_type'].isin(['SINGLE_SALE_OFFER', 'ENGLISH_AUCTION', 'DIRECT_OFFER', 'SINGLE_BUY_OFFER', 'INSTANT_BUY']),
    la_liga_transactions_df['transaction_type'].isin(['REWARD', 'REFERRAL'])
]
values = ['paid_transaction', 'free_transaction']
default_value = 'other'
la_liga_transactions_df['transaction_category'] = np.select(conditions, values, default=default_value)

# Create the transaction_market column
market_conditions = [
    la_liga_transactions_df['transaction_type'].isin(['ENGLISH_AUCTION', 'INSTANT_BUY']),
    la_liga_transactions_df['transaction_type'].isin(['SINGLE_SALE_OFFER', 'DIRECT_OFFER', 'SINGLE_BUY_OFFER'])
]
market_values = ['Primary', 'Secondary']
market_default_value = 'not-applicable'
la_liga_transactions_df['transaction_market'] = np.select(market_conditions, market_values, default=market_default_value)


In [44]:
la_liga_transactions_df['transaction_category'].value_counts()

transaction_category
paid_transaction    1136499
free_transaction     121421
other                  3094
Name: count, dtype: int64

In [45]:
#Handle with the trade of cards (direct_offers that appears with price = 0)

# Rename the category from "paid_transaction" to "monetary_transaction"
la_liga_transactions_df['transaction_category'] = la_liga_transactions_df['transaction_category'].replace('paid_transaction', 'monetary_transaction')

# Create the new category "trade"
trade_condition = (la_liga_transactions_df['transaction_type'] == 'DIRECT_OFFER') & (la_liga_transactions_df['price_eur'] == 0)
la_liga_transactions_df.loc[trade_condition, 'transaction_category'] = 'trade'


In [46]:
la_liga_transactions_df['transaction_category'].value_counts()

transaction_category
monetary_transaction    940273
trade                   196226
free_transaction        121421
other                     3094
Name: count, dtype: int64

##### Flagging of abnormal transactions
In some cases (misprint or personal arrangements (loans, etc)) a out-of-market price will intervene. To flag in a very volatile market, we can calculate a rolling window statistic, such as the mean or median and flag the transactions that differs a lot from that. to be implemented if time 


#### i. Flagging current / precedent season card.
more and more elements in the gamemplay benefit more to the current season cards compared to the ones printed in the previous seasons. So the price impact may have evolve on that point. 
NB: A major gameplay change has intervene the 7th of February where it is a key element. Maybe it can be seen in the very last days of evolution but it will be a key if running an analysis in one month. 

In [47]:
la_liga_transactions_df.head(2)

,present_team_name,card_id,asset_id,card_name,season_year,card_slug,rarity,card_creation_date,grade,power,...,team_slug,2021_22_la_liga,2022_23_la_liga,2023_24_la_liga,2021_22_uefa,2022_23_uefa,2023_24_uefa,price_eth,transaction_category,transaction_market
0,Real Madrid CF,Card:00035848-b60d-4999-832c-e4dd8e939007,0x040004752c83837b25107500c71efa485aafb6cf1b82...,Nacho 2021-22 • Limited 218/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-218,limited,2021-11-25 03:40:39+00:00,7,1.045,...,real-madrid-madrid,True,True,True,True,True,True,0.0000,free_transaction,not-applicable
1,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31 08:58:22+00:00,13,1.075,...,real-madrid-madrid,True,True,True,True,True,True,0.0065,monetary_transaction,Primary


In [48]:
la_liga_transactions_df['in_season_card_transaction'] = la_liga_transactions_df['transaction_date_season'].astype(str).str.replace('season_', '') == la_liga_transactions_df['season_year'].astype(str)

In [49]:
la_liga_transactions_df['in_season_card_transaction'].value_counts()

in_season_card_transaction
True     895755
False    365259
Name: count, dtype: int64

#### Creation of Demand & Supply indicators: 

In [50]:
### For user balance: users with first buy & User with last sale 
### to see later if time


In [51]:
### Differenciation Manager/Trader

In [52]:
la_liga_transactions_df['user_slug'].nunique()

141341

In [53]:
top_100_user_transac_count = la_liga_transactions_df['user_slug'].value_counts().head(100).index
la_liga_transactions_df['user_category'] = np.where(la_liga_transactions_df['user_slug'].isin(top_100_user_transac_count), 'trader', 'manager')

In [54]:
#Creating a sub-dataset only on first transaction to ease some data analysis: 

la_liga_transactions_df = la_liga_transactions_df.sort_values('transaction_date')
first_la_liga_trans_df = la_liga_transactions_df.groupby('card_id').first().reset_index()
la_liga_cards_df = first_la_liga_trans_df[['card_id', 'asset_id', 'card_name', 'season_year', 'card_slug', 'rarity', 'card_creation_date', 'grade', 'power', 'xp', 'serial_number', 'shirt_number', 'special_edition', 'player_slug', 'card_creation_date_year', 'card_creation_date_month', 'card_creation_date_week', 'card_creation_date_dayofyear', 'card_creation_date_season', 'position', 'age', 'u23_eligible', 'u23_until', 'club_slug_upon_transaction']].copy()

In [55]:
la_liga_cards_df.shape

(474355, 24)

In [56]:
only_transactions_df = la_liga_transactions_df[['card_id', 'transaction_date', 'transaction_type', 'price_wei', 'price_eur', 'price_eth', 'user_id', 'user_slug', 'player_slug', 'transaction_date_year', 'transaction_date_month', 'transaction_date_week', 'transaction_date_dayofyear', 'transaction_date_season', 'club_slug_upon_transaction', 'transaction_category', 'transaction_market', 'in_season_card_transaction', 'user_category']].copy()

In [57]:
#la_liga_transactions_df.to_csv('la_liga_filtered_cards_transactions.csv', index=False)
#only_transactions_df.to_csv('transactions_only.csv',index=False)
#la_liga_cards_df.to_csv('la_liga_cards.csv',index=False)

#### Export to MyQL

In [58]:
pw = os.getenv('MySQLpwd')
connection_string = 'mysql+pymysql://root:' + pw + '@localhost:3306/'
engine = create_engine(connection_string)

In [59]:
clubs_by_season_df.to_sql(name='clubs',con=engine, schema='final_project', if_exists='replace', index=False)

24

In [60]:
#la_liga_cards_df.to_sql(name='cards',con=engine, schema='final_project', if_exists='replace', index=False)

474355

In [62]:
only_transactions_df.dtypes

card_id                                    object
transaction_date              datetime64[ns, UTC]
transaction_type                           object
price_wei                                   int64
price_eur                                 float64
price_eth                                 float64
user_id                                    object
user_slug                                  object
player_slug                                object
transaction_date_year                       int32
transaction_date_month                      int32
transaction_date_week                      UInt32
transaction_date_dayofyear                  int32
transaction_date_season                    object
club_slug_upon_transaction                 object
transaction_category                       object
transaction_market                         object
in_season_card_transaction                   bool
user_category                              object
dtype: object

In [63]:
#only_transactions_df.to_sql(name='transactions',con=engine, schema='final_project', dtype={'transaction_date':sqlalchemy.DateTime()}, if_exists='replace', index=False)

1261014

In [64]:
players_club_hist_df['u23_until'] = players_club_hist_df['u23_until'].fillna('NA')

In [82]:
max(cleaned_players_club_hist_df['club_slug'].apply(len))

37

In [71]:
cleaned_players_club_hist_df = players_club_hist_df[players_club_hist_df['club_slug'].isin(clubs_by_season_df['team_slug'])]

In [73]:
cleaned_players_club_hist_df.to_sql(name='clubs_by_players',con=engine, schema='final_project', dtype={'contract_start_date':sqlalchemy.DateTime(),'contract_end_date':sqlalchemy.DateTime()},if_exists='replace', index=False)

2423

## Data Cleaning and Wrangling of the Players Scores Dataframe

In [74]:
players_scores_df = pd.read_csv('la_liga_players_scores.csv')

In [75]:
players_scores_df.shape

(121275, 13)

In [79]:
players_scores_df.isna().sum()

player_name                       0
player_slug                       0
player_id                         0
age                               0
u23_eligible                      0
u23_until                         0
so5_score                         0
decisive_score                    0
gameweek_date                     0
gameweek_number                   0
gameweek_id                       0
rolling_avg_last_5_scores     44333
rolling_avg_last_15_scores    44333
dtype: int64

In [78]:
players_scores_df['u23_until'] = players_scores_df['u23_until'].fillna('NA')

In [93]:
players_scores_df['gameweek_date'] = pd.to_datetime(players_scores_df['gameweek_date'])

In [94]:
players_scores_df.to_sql(
    name='temp_players_scores',
    con=engine,
    schema='final_project',
    dtype={'gameweek_date': sqlalchemy.DateTime()},
    if_exists='replace',
    index=False
)

121275

In [87]:

sorted_gameweek_numbers = players_scores_df['gameweek_number'].sort_values()
distinct_gameweek_numbers = sorted_gameweek_numbers.unique()

print(distinct_gameweek_numbers)

[ 20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  51  52  53  54  55  56
  58  59  60  61  62  63  64  66  67  68  69  70  71  72  73  74  75  76
  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94
  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112
 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130
 131 132 133 134 135 136 137 138 139 140 142 143 144 145 146 147 148 149
 150 151 152 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168
 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186
 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204
 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222
 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240
 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258
 259 260 261 262 263 264 265 266 267 268 269 270 27

In [90]:

memphis_scores=players_scores_df[players_scores_df['player_slug']=='memphis-depay']
sorted_gameweek_memphis = memphis_scores['gameweek_number'].sort_values()
distinct_gameweek_numbers = sorted_gameweek_memphis.unique()


In [88]:
memphis_scores

,player_name,player_slug,player_id,age,u23_eligible,u23_until,so5_score,decisive_score,gameweek_date,gameweek_number,gameweek_id,rolling_avg_last_5_scores,rolling_avg_last_15_scores
16789,Memphis Depay,memphis-depay,Player:95e34a9f-d308-4abf-b432-9bf89f68577c,29,False,NA,60.0,60.0,2024-02-04T20:00:00Z,447,So5Fixture:73a2e71f-84ad-49dc-aa3e-2105453584b1,60.000000,60.000000
16790,Memphis Depay,memphis-depay,Player:95e34a9f-d308-4abf-b432-9bf89f68577c,29,False,NA,69.3,60.0,2024-01-31T20:00:00Z,446,So5Fixture:86b5d805-2f4d-4e3d-8ffc-1ef7b4513357,64.650000,64.650000
16791,Memphis Depay,memphis-depay,Player:95e34a9f-d308-4abf-b432-9bf89f68577c,29,False,NA,71.5,60.0,2024-01-28T20:00:00Z,445,So5Fixture:c41692a5-c060-4a83-9eba-f53f99c66899,66.933333,66.933333
16792,Memphis Depay,memphis-depay,Player:95e34a9f-d308-4abf-b432-9bf89f68577c,29,False,NA,0.0,25.0,2024-01-22T20:00:00Z,443,So5Fixture:f20d1b70-b81c-4d82-94d9-1ee4067539e3,NaN,NaN
16793,Memphis Depay,memphis-depay,Player:95e34a9f-d308-4abf-b432-9bf89f68577c,29,False,NA,0.0,25.0,2024-01-10T19:00:00Z,440,So5Fixture:e1f8f0de-65c6-4a04-be99-e2eb921c0aef,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16982,Memphis Depay,memphis-depay,Player:95e34a9f-d308-4abf-b432-9bf89f68577c,29,False,NA,55.5,35.0,2019-09-28T11:30:00Z,27,So5Fixture:41b7318e-a00c-42df-a6b3-ad4d5b141c96,80.040000,56.673333
16983,Memphis Depay,memphis-depay,Player:95e34a9f-d308-4abf-b432-9bf89f68577c,29,False,NA,41.0,35.0,2019-09-22T19:00:00Z,26,So5Fixture:d4f9f578-51b4-4ee7-9ab1-7bdb2e2bb0fb,68.240000,57.140000
16984,Memphis Depay,memphis-depay,Player:95e34a9f-d308-4abf-b432-9bf89f68577c,29,False,NA,75.7,60.0,2019-08-31T15:30:00Z,24,So5Fixture:38c0299c-d6dc-4bd8-8d10-061fe2db3ab8,63.380000,58.400000
16985,Memphis Depay,memphis-depay,Player:95e34a9f-d308-4abf-b432-9bf89f68577c,29,False,NA,100.0,80.0,2019-08-16T18:45:00Z,22,So5Fixture:58cad14d-80c4-4782-92a6-aa6ef863e5b3,63.380000,59.620000


In [ ]:
#create a column with null for 0 